In [15]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [16]:
# Загрузка данных
x_train = np.load('datasets/dataset3/x_train.npy')
y_train = np.load('datasets/dataset3/y_train.npy')

x_test = np.load('datasets/dataset3/x_test.npy')
y_test = np.load('datasets/dataset3/y_test.npy')

print(x_train[:5])
print(y_train)

[[-0.8123405  -0.64663589  1.17611488  1.5075241   0.84470565 -0.8123405
  -0.8123405  -0.8123405  -0.8123405   0.01618257  1.83893333  1.83893333
   1.83893333 -0.14952204 -0.8123405  -0.8123405  -0.8123405  -0.8123405
  -0.8123405  -0.14952204  1.83893333 -0.14952204 -0.8123405  -0.8123405
  -0.8123405  -0.8123405  -0.8123405   0.3475918   1.67322872 -0.64663589
  -0.8123405  -0.8123405  -0.8123405  -0.8123405  -0.48093127  1.67322872
   0.67900103 -0.8123405  -0.8123405  -0.8123405  -0.8123405  -0.8123405
   0.67900103  1.67322872 -0.48093127 -0.8123405  -0.8123405  -0.8123405
  -0.8123405  -0.8123405   1.83893333  1.5075241   0.51329642  0.51329642
  -0.31522666 -0.8123405  -0.8123405  -0.8123405   0.84470565  1.67322872
   1.34181949  0.67900103 -0.14952204 -0.8123405 ]
 [-0.8123405  -0.8123405  -0.8123405  -0.8123405   0.51329642  1.5075241
   0.84470565 -0.8123405  -0.8123405  -0.8123405  -0.8123405   0.67900103
   0.3475918   0.67900103  1.17611488 -0.8123405  -0.8123405  -0.81

In [17]:
class SelfmadeLogisticRegression:
    def __init__(self, learning_rate=0.01, n_iterations=1000, reg_strength=0.01):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.reg_strength = reg_strength
        self.models = {}  # Словарь моделей для каждого класса

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def log_loss(self, y, predictions):
        epsilon = 1e-15
        predictions = np.clip(predictions, epsilon, 1 - epsilon)
        loss = - (y * np.log(predictions) + (1 - y) * np.log(1 - predictions))
        return np.mean(loss)

    def fit(self, X, y):
        unique_classes = np.unique(y)

        for cls in unique_classes:
            binary_y = (y == cls).astype(int)  # Преобразуем к бинарной классификации
            model = self.train_binary_model(X, binary_y)
            self.models[cls] = model

    def train_binary_model(self, X, y):
        n_samples, n_features = X.shape
        weights = np.zeros(n_features)
        bias = 0

        for _ in range(self.n_iterations):
            model = np.dot(X, weights) + bias
            predictions = self.sigmoid(model)

            dw = (1 / n_samples) * (np.dot(X.T, (predictions - y)) + self.reg_strength * weights)
            db = (1 / n_samples) * np.sum(predictions - y)

            weights -= self.learning_rate * dw
            bias -= self.learning_rate * db

        return weights, bias

    def predict(self, X):
        predictions = np.zeros((X.shape[0], len(self.models)))

        for cls, model in self.models.items():
            weights, bias = model
            model_predictions = np.dot(X, weights) + bias
            predictions[:, int(cls)] = self.sigmoid(model_predictions)

        # Выбираем класс с наибольшей вероятностью
        return np.argmax(predictions, axis=1)

In [18]:
# Инициализация модели
model = LogisticRegression(
    solver='liblinear',
    C=100.0,
    random_state=0
)

# Обучение модели
model.fit(x_train, y_train)

# Предсказание на тестовых данных
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)

print(f'Точность модели: {accuracy * 100:.2f}%')
print(y_pred)

Точность модели: 96.44%
[8 2 1 0 6 4 8 7 6 0 9 0 3 7 5 9 2 9 6 6 2 2 4 0 0 3 4 9 2 3 0 2 2 9 9 5 2
 0 0 5 7 6 3 3 1 8 7 3 5 0 1 4 2 0 8 9 1 7 1 4 9 1 0 1 7 9 2 1 4 9 2 7 5 2
 9 8 6 3 0 6 0 1 3 0 2 2 6 1 5 8 8 1 6 3 2 7 7 3 0 9 9 9 7 6 0 2 0 8 1 1 9
 2 0 7 1 5 2 2 3 1 2 0 3 5 2 2 3 8 6 5 8 3 7 7 2 1 0 7 3 6 1 7 6 3 3 6 2 9
 7 5 3 0 7 0 8 8 0 6 5 1 5 9 5 3 4 7 0 4 7 9 5 5 0 9 9 8 4 1 1 4 3 7 1 4 5
 0 7 1 0 5 8 7 0 8 6 2 9 7 6 3 1 8 9 3 6 9 3 4 0 5 6 3 0 2 1 2 3 1 9 4 2 2
 9 6 4 6 1 0 7 5 0 3 9 1 3 9 6 2 8 9 5 8 1 2 5 9 2 4 1 6 0 1 3 2 2 4 3 5 3
 4 4 3 2 1 9 6 5 4 7 2 0 7 2 7 2 7 5 7 5 4 9 8 2 6 2 5 8 0 4 1 6 6 6 8 4 8
 2 0 2 8 2 8 9 5 3 6 0 8 8 0 9 5 7 7 4 6 5 9 5 5 7 3 7 2 3 6 7 4 1 5 0 8 9
 1 3 5 8 6 1 0 1 7 5 7 3 6 5 6 4 3 4 0 5 8 4 1 2 5 0 3 3 7 6 5 9 9 3 8 3 6
 5 5 5 5 5 6 6 5 2 3 8 7 8 9 5 9 7 5 9 5 6 4 3 3 3 9 1 4 2 2 2 9 4 0 3 7 5
 7 6 5 0 9 2 6 9 8 7 2 9 8 4 0 4 0 5 7 4 4 7 3 0 7 4 8 7 1 5 2 6 2 9 7 9 5
 6 1 5 7 8 4]


In [19]:
# Используем класс
selfmade_model = SelfmadeLogisticRegression()
selfmade_model.fit(x_train, y_train)
selfmade_predictions = selfmade_model.predict(x_test)

print(selfmade_predictions)

[1 2 2 0 6 4 8 7 6 0 9 0 3 7 5 9 2 9 6 6 2 2 4 0 0 3 4 9 2 3 0 2 2 9 9 5 1
 0 0 5 7 6 3 3 8 8 7 3 5 0 8 4 2 0 8 8 1 7 1 4 9 1 0 9 7 9 2 8 8 9 0 7 5 2
 9 8 6 3 0 6 0 1 3 0 2 2 6 6 5 8 8 1 6 9 2 7 7 3 0 9 9 9 7 6 0 2 0 8 1 1 9
 2 0 7 1 5 3 2 3 1 2 0 3 5 2 2 3 8 6 5 8 3 7 7 2 1 0 7 3 6 1 7 6 3 3 1 2 9
 7 5 3 0 7 0 8 8 0 6 5 1 5 9 5 3 4 7 0 4 7 9 5 5 0 9 9 8 4 1 1 4 3 7 1 4 5
 0 7 1 0 5 8 7 0 8 6 2 9 7 6 3 1 8 9 7 6 9 3 4 0 5 6 3 0 2 1 2 3 1 9 4 2 2
 9 6 4 6 1 0 7 5 0 8 9 1 3 9 6 2 8 9 5 8 1 2 5 9 2 4 9 6 0 1 3 2 2 4 3 5 3
 4 4 3 1 1 5 6 0 4 9 2 0 7 2 9 2 7 5 7 5 4 9 8 2 6 2 5 8 0 4 1 6 6 4 1 4 8
 2 0 2 8 2 8 9 5 3 6 0 9 7 0 7 5 7 7 4 6 5 9 5 5 7 3 7 2 3 6 7 4 9 5 0 8 9
 1 3 5 8 6 1 0 1 7 5 7 3 6 5 6 4 2 4 0 5 9 4 1 3 9 0 3 3 7 6 5 9 9 3 8 3 6
 5 5 5 5 5 6 6 5 7 3 8 7 8 9 5 9 7 5 7 5 6 4 3 3 3 9 1 4 2 2 2 9 4 0 3 7 5
 7 6 5 0 9 2 6 9 8 7 1 9 8 4 0 4 0 5 7 4 4 7 3 0 7 4 1 7 8 5 2 6 8 9 7 9 5
 6 1 5 7 8 4]


In [20]:
# Сравнение результатов
selfmade_accuracy = accuracy_score(y_test, selfmade_predictions)

In [21]:
print(f'Точность самописной модели: {selfmade_accuracy * 100:.2f}%')

Точность самописной модели: 91.56%
